<a href="https://colab.research.google.com/github/whitebearovo/Android-Builder/blob/main/sakura_14b_qwen2_5_v1_0_q6k_gguf_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title # Keep this widget playing to prevent Colab from disconnecting you { display-mode: "form" }
#@markdown Press play on the audio player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# Mount Google Drive
Mount_GDrive = True # @param {type:"boolean"}
if Mount_GDrive:
  from google.colab import drive
  drive.mount('/content/gdrive')
  ROOT_PATH = "/content/gdrive/MyDrive"
else:
  ROOT_PATH = "/content"
!nvidia-smi

In [ ]:
# Install dependencies
!pip install --upgrade pip
!pip install transformers tokenizers
!pip install vllm huggingface-hub flask pyngrok triton torch

In [ ]:
# Run API backend (vLLM)
# Use ngrok for API mapping
from huggingface_hub import hf_hub_download

# Set the ngrok authentication token
ngrokToken = "2pR573GJUUUuRlAPKFQcKemTJk7_2uisJCJJ8ng8ERcwBexTp"

if ngrokToken:
    from pyngrok import conf, ngrok

    # Configure the ngrok authentication token
    conf.get_default().auth_token = ngrokToken
    conf.get_default().monitor_thread = False

    # Start ngrok tunnel with the custom domain
    try:
        ssh_tunnel = ngrok.connect(8001, bind_tls=True, hostname="devoted-hen-awaited.ngrok-free.app")
        public_url = ssh_tunnel.public_url
        print('Custom Domain Address: ' + public_url)
    except Exception as e:
        print(f"Error starting ngrok tunnel: {e}")
else:
    # Download cloudflared tunnel, use cloudflare to set up temporary tunnel (Not Static URL)
    %cd $ROOT_PATH

    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
    !chmod a+x cloudflared
    !./cloudflared tunnel --url localhost:8001




%cd $ROOT_PATH

# Download the model for the first time
!HF_ENDPOINT=https://huggingface.co huggingface-cli download SakuraLLM/Sakura-14B-Qwen2.5-v1.0-GGUF --local-dir models --include sakura-14b-qwen2.5-v1.0-q6k.gguf

!RAY_memory_monitor_refresh_ms="0" HF_ENDPOINT=https://huggingface.co OMP_NUM_THREADS=36 VLLM_ATTENTION_BACKEND=XFORMERS vllm serve ./models/sakura-14b-qwen2.5-v1.0-q6k.gguf --tokenizer Qwen/Qwen2.5-14B-Instruct --dtype float16 --api-key token-abc123 --kv-cache-dtype auto --max-model-len 4096 --tensor-parallel-size 1 --gpu-memory-utilization 0.99 --disable-custom-all-reduce --enforce-eager --use-v2-block-manager --disable-log-requests --host 0.0.0.0 --port 8001 --served-model-name "Qwen2.5-14B-Instruct" &

